# Accessing OBIS data

OBIS holds more than 136 million records, across >5,000 datasets, and with >181 million measurements. This huge amount of data is driving research across a diverse range of topics.

Here we will explore how to access OBIS data through **R**, but all the operations can also be done through **Python**. Check the [`pyobis` module]() and the other notebooks in this repo. We will focus on getting the different types of data, rather than on its direct application/integration with remote sensing, although a few examples will be provided.

----

`robis` is the main gateway for OBIS data through R. You can learn more about `robis` in the [OBIS manual.](https://manual.obis.org/access.html#r-package)

Hands-on outline:

1. Downloading occurrence records 
2. Get species lists 
3. Extended measurements
4. Events and time series

In a second notebook, we will explore two other products:

1. Full export 
2. Gridded product 

Before starting, we need to install a few packages on Google Colab (if you are using Binder, just skip this cell):

In [ ]:
# On your local computer you can use install.packages() for everything
# Here we use the system interface to some, as the installation
# with r2u is faster.
system("apt-get install r-cran-sf r-cran-terra") # Spatial packages
system("apt-get install r-cran-robis")           # OBIS API interface
system("apt-get install r-cran-arrow")           # To deal with full export
system("apt-get install r-cran-DBI")             # To deal with full export
system("apt-get install r-cran-duckdb")          # To deal with full export
system("apt-get install r-cran-rnaturalearth")   # For mapping
system("apt-get install r-cran-h3jsr")           # For indexing (H3 sytem)

# Install additional from GitHub
system("apt-get install r-cran-bspm")
bspm::enable()
devtools::install_github("iobis/obistools")
devtools::install_github("ropensci/mregions2")

### 1. Downloading occurrence records

To download records from OBIS we use the function `occurrence`. There are many arguments you can pass to download data for a specific species, taxonomic level or region.

We will start by getting data for three taxonomic entities:

<div style="display: flex; flex-direction: row; max-height: 200px; padding: 5px;">
<div>
<p>Species: <i>Thalassia testudinum</i></p><img src="https://upload.wikimedia.org/wikipedia/commons/d/da/Thalassia_testudinum_%28turtle_grass%29_%28South_Pigeon_Creek_estuary%2C_San_Salvador_Island%2C_Bahamas%29_6_%2815859999657%29.jpg" height=200 alt="Source: https://upload.wikimedia.org/wikipedia/commons/d/da/Thalassia_testudinum_%28turtle_grass%29_%28South_Pigeon_Creek_estuary%2C_San_Salvador_Island%2C_Bahamas%29_6_%2815859999657%29.jpg"></img>
</div>
<div>
<p>Genus: <i>Thalassia</i></p><img src="https://upload.wikimedia.org/wikipedia/commons/2/2e/Thalassia_Hemprichii.jpg" height=200 alt="Source: https://upload.wikimedia.org/wikipedia/commons/2/2e/Thalassia_Hemprichii.jpg"></img>
</div>
<div>
<p>Order: Alismatales</p><img src="https://upload.wikimedia.org/wikipedia/commons/5/55/Eelgrass_%28Zostera_marina%29_-_iNaturalist.org.jpg" height=200 alt="Source: https://upload.wikimedia.org/wikipedia/commons/5/55/Eelgrass_%28Zostera_marina%29_-_iNaturalist.org.jpg"></img>
</div>
</div>

In [ ]:
library(robis)
library(dplyr)

thal_test <- occurrence("Thalassia testudinum")

nrow(thal_test)
head(thal_test)

We can make a map of the records. For that, we can use a handy function from the `obistools` package.

In [ ]:
library(obistools)

plot_map(thal_test)

OBIS is different in that it was created and is optimized for marine data. It matches its taxonomic names (the species identity) with the World Register of Marine Species (WoRMS). Each species (or any other taxonomic level) has a unique ID called **AphiaID**. You can see for example the entry for _Thalassia testudinum_ [here.](https://www.marinespecies.org/aphia.php?p=taxdetails&id=374720)

This is important because species names can change over time, in case a species is reclassified into another group or when we discover that one species can be in fact multiple species.

The `occurrence` function also accepts an argument named `taxonid`. So, the same we did on the first cell, can be done with:

In [ ]:
# This is the same as
# thal_test <- occurrence("Thalassia testudinum")
thal_test <- occurrence(taxonid = 374720)

Let's try the other taxonomic levels:

In [ ]:
# The genus
thal_genus <- occurrence("Thalassia")

nrow(thal_genus)
table(thal_genus$scientificName)

# The order. Here we will limit by records from 2020 onwards, to speed up
# the download
alismatales <- occurrence("Alismatales", startdate = "2020-01-01")

nrow(alismatales)
table(alismatales$scientificName)

Records are organized in **datasets** that group data that was collected in a particular survey, study, monitoring, etc. We can get additional information about datasets using the `dataset` function (which can also be used to list all datasets for a specific `scientificName`).

In [ ]:
# Get the number of records by dataset
thal_test_ds <- thal_test |> 
    group_by(dataset_id) |> 
    summarise(records = n())

high_n <- thal_test_ds[order(thal_test_ds$records, decreasing = T), "dataset_id"][1,]

ds_info <- dataset(datasetid = high_n$dataset_id[1])

head(ds_info[,1:5])

In [ ]:
# Get all datasets for Thalassia testudinum
ds_thal <- dataset(scientificname = "Thalassia testudinum")
nrow(ds_thal)
ds_thal |> select(id, url, core, records, node_name) |> head()

It is also possible to download data for a specific area. There are two ways. The first is to use the OBIS regions. You can explore it using the mapper, for example this one: https://mapper.obis.org/?areaid=34288

<img src="https://upload.wikimedia.org/wikipedia/commons/a/a6/Gulf_of_Mexico_topographic_location_map.png" height=200 alt="Source: https://upload.wikimedia.org/wikipedia/commons/a/a6/Gulf_of_Mexico_topographic_location_map.png"></img>

In [ ]:
# If you pass only the areaid as below it will download all the data for that
# area. In this case, it is a huge amount (3,141,354 records)! So, let's not try.
# gulf <- occurrence(areaid = 27)
# Instead, we will get all records of Thalassia testudinum here:
gulf_seagrass <- occurrence(areaid = 34288, scientificname = "Thalassia testudinum")

head(gulf_seagrass)

Another approach is to pass a geometry in Well-known Text (WKT). You can draw polygons in this website: https://wktmap.com/

In [ ]:
# Let's get data for the Florida region, regarding some seagrass species:
wkt_area <- "POLYGON ((-84.353027 23.180764, -77.980957 23.180764, -77.980957 28.22697, -84.353027 28.22697, -84.353027 23.180764))"

# We can check it by plotting
wrld <- rnaturalearth::ne_countries(returnclass = "sf")
wkt_sf <- sf::st_as_sfc(wkt_area)
terra::plot(terra::vect(wrld[,1]), xlim = c(-87, -74), ylim = c(21, 29), col = "grey70")
terra::lines(terra::vect(wkt_sf), col = "blue")

In [ ]:
occ_area <- occurrence(
    scientificname = "Alismatales",
    geometry = wkt_area
)

nrow(occ_area)

plot_map(occ_area, zoom = T)

OBIS has occurrence information, but also absence data. The extent to which those absences can be considered to be a "true" absence depends on the methods used to collect the data. There are many datasets that were collected using standard methods, and for which the absences can be considered of a good quality. Let's run again the previous code, but with the option `absence = "include"`

In [ ]:
occ_area <- occurrence(
    scientificname = "Alismatales",
    geometry = wkt_area,
    absence = "include"
)

In [ ]:
occ_area |> 
    group_by(dataset_id, absence) |> 
    count() |> 
    arrange(desc(n)) |> 
    head()


We see that there are two datasets with a great number of absences. Let's explore the second one, which is the ["South Florida Fisheries Habitat Assessment Program (FHAP-SF)"](https://obis.org/dataset/04326c80-e971-4110-9884-20f843ee73ba).

We will just filter the previous data, but you can download the data for only this dataset by using:

`occurrence(datasetid = "04326c80-e971-4110-9884-20f843ee73ba")`

In [ ]:
fhap_ds <- occ_area |> 
    filter(dataset_id == "04326c80-e971-4110-9884-20f843ee73ba")

table(fhap_ds$absence)

terra::plot(terra::vect(wrld[,1]), xlim = c(-82, -80), ylim = c(24, 26), col = "grey70")
terra::lines(terra::vect(wkt_sf), col = "blue")
points(fhap_ds[,c("decimalLongitude", "decimalLatitude")],
        col = as.factor(fhap_ds$absence), pch = 20, cex = .2)

This type of information can be used to ground-truth maps or models, or to train a model. We will show a simple (and not scientifically accurate) example of a classification model using this data.

### 2. Get species lists 

Sometimes we are only interested in knowing which species are present in a region. This information, called a checklist, can be easily obtained through the function `robis::checklist()`. Let's try with another region, a polygon covering the Gulf of Finland.

In [ ]:
gulf_fin <- "POLYGON ((21.247559 57.984808, 29.443359 57.984808, 29.443359 61.522695, 21.247559 61.522695, 21.247559 57.984808))"

gulf_fin_check <- checklist(geometry = gulf_fin)

head(gulf_fin_check)

Again, you can make further filtering when calling the function. For example, only Clorophyta recorded between 2000 and 2010.

In [ ]:
gulf_fin_chl_check <- checklist(geometry = gulf_fin,
                                scientificname = "Chlorophyta",
                                startdate = "2000-01-01",
                                enddate = "2010-12-01")

head(gulf_fin_chl_check)

gulf_fin_chl_check |> 
    select(scientificName, records) |> 
    head(3)

### 3. Extended measurements

OBIS is much more than just occurrence records. There are >180 million measurements that you can also explore using the API. Measurements are made available through the **MeasurementsOrFacts** extension. We are currently seeking ways to improve the access and discoverability of this data. For now, we admit, it is a bit complicated to explore it.

A good way is to first check which datasets in your area of interest and target group have the extension:

In [ ]:
gulf_fin_ds <- dataset(geometry = gulf_fin,
                       scientificname = "Chlorophyta",
                       hasextensions = "MeasurementOrFact")

str(gulf_fin_ds$extensions)

We see that all of them have the `measurementorfact` extension, and two have also the `dnaderived` extension.

You can see it on OBIS:

- With eMoF and occurrence: https://obis.org/dataset/057d7f8c-5286-4032-aaf9-5ce078833262
- With eMoF and dnaderived: https://obis.org/dataset/066f002f-58d5-4687-bdb8-b39cdaef0c2b

If you scroll down, you will see a section called _Measurement types_, where you can check which measurements are available.

As an example, we will load the first dataset, which has interesting measurements of phytoplankton. To get the eMoF, we will now set `mof = TRUE` in the `occurrence` function. Because it is a long time series, we will just get recent data to save time.

In [ ]:
kplank_ds <- occurrence(
    datasetid = "057d7f8c-5286-4032-aaf9-5ce078833262",
    mof = TRUE,
    startdate = "2020-01-01"
)

nrow(kplank_ds)
head(kplank_ds)

This dataset now contains a column called `mof` which is made of lists.

In [ ]:
str(kplank_ds$mof[1])

The name of the columns follow the guidelines of the extended Measurements or Facts (eMoF) extension. You can learn more about it [in the OBIS manual.](https://manual.obis.org/format_emof.html) To use that data we need to "unnest" it, that is, convert to a `data.frame`. We will use the function `unnest_extension`

In [ ]:
# Note: this is quite memory intensive and may take some time to process
kplank_ext <- unnest_extension(df = kplank_ds,
                               extension = "MeasurementOrFact",
                               fields = "eventDate")

head(kplank_ext)

table(kplank_ext$measurementType)

Many measurements are available. We will check now only the "Carbon biomass of phytoplankton per unit volume of the water body".

In [ ]:
carbon_biomass <- kplank_ext |> 
    filter(measurementType == "Carbon biomass of phytoplankton per unit volume of the water body")

head(carbon_biomass, 3)

Let's say that we want to produce a simple map of carbon biomass to later compare with a satellite product or to a model output. First, we need to get the longitude and latitude from the occurrence records and merge it with this table. Then, we group by longitude, latitude and date to get the total carbon in that place and date.

In [ ]:
carbon_biomass <- carbon_biomass |> 
    left_join(kplank_ds[,c("occurrenceID", "decimalLongitude", "decimalLatitude")]) |> 
    mutate(eventDate = as.Date(eventDate)) |> 
    group_by(eventDate, decimalLongitude, decimalLatitude) |> 
    filter(!is.na(measurementValue)) |> 
    mutate(measurementValue = as.numeric(measurementValue)) |> 
    summarise(total_carbon = sum(measurementValue)) |> 
    ungroup() |> 
    group_by(decimalLongitude, decimalLatitude) |> 
    mutate(site_id = cur_group_id())

head(carbon_biomass, 3)

Let's plot to see.

In [ ]:
library(ggplot2)
library(sf)
library(terra)
sf_use_s2(FALSE)

carbon_biomass_sf <- st_as_sf(carbon_biomass, 
                              coords = c("decimalLongitude", "decimalLatitude"), 
                              crs = "EPSG:4326")
wrld_cr <- st_crop(wrld, carbon_biomass_sf)

ggplot() +
    geom_sf(data = wrld_cr, fill = "grey70") +
    geom_sf(data = carbon_biomass_sf, aes(size = total_carbon, color = total_carbon), alpha = .5) +
    scale_color_viridis_c() +
    coord_sf() + theme_light()


We will use the data as an aggregate, ignoring the time component:

In [ ]:
carbon_gam <- mgcv::gam(total_carbon ~ s(decimalLongitude,
                                         decimalLatitude, 
                                         bs = 'gp', k = 100,
                                         m = 2),
                        data = carbon_biomass)
summary(carbon_gam)

In [ ]:
grid_pred <- rast(ext(wrld_cr), res = 0.1)
grid_pred[] <- 1
grid_pred_df <- as.data.frame(grid_pred, xy = T)[,1:2]
colnames(grid_pred_df) <- c("decimalLongitude", "decimalLatitude")

pred_carbon <- predict(carbon_gam, grid_pred_df)

grid_pred_df$carbon <- pred_carbon

ggplot() +
    geom_raster(data = grid_pred_df, aes(x = decimalLongitude, y = decimalLatitude, fill = carbon)) +
    scale_fill_viridis_c() +
    geom_sf(data = wrld_cr, fill = "grey70") +
    geom_sf(data = carbon_biomass_sf, aes(size = total_carbon), color = "white", fill = NA, alpha = .1, shape = 1) +
    coord_sf() + theme_light()

### 4. Events and time series

The dataset we have just used is a good example of a time series, coming from a long-term monitoring. Data in OBIS is usually organized in using Event Core (you can learn more [here](https://manual.obis.org/formatting.html#when-to-use-event-core)), as this is used to organize data when there are more than one sampling occasion and/or location, and different occurrences linked to each sampling (as most ecological studies). However, not all events form a time series! In many cases, you might have a single or few surveys. See for example the dataset below:

In [ ]:
poly_med <- occurrence(datasetid = "727322b7-bdfd-4229-bb10-850ff97b5833")

head(poly_med, 2)

table(poly_med$eventID, poly_med$eventDate)

There are 2 events, all from the same date. In many cases you might be interested in finding time series. Here is an example of how you can do that:

In [ ]:
# Define an area of interest, in that case a small section of the Mediterranean
wkt_area <- "POLYGON ((10.107422 37.68382, 15.512695 37.68382, 15.512695 41.244772, 10.107422 41.244772, 10.107422 37.68382))"

occ_area <- occurrence(
    geometry = wkt_area,
    # We will focus on fishes
    scientificname = "Teleostei",
    startdate = "2010-01-01"
)

occ_area_ts <- occ_area |> 
    group_by(dataset_id, 
             date_year,
             decimalLongitude = round(decimalLongitude, 3),
             decimalLatitude = round(decimalLatitude, 3)) |> 
    distinct() |> 
    summarise(total = n()) |> 
    ungroup() |> 
    filter(total >= 3)

nrow(occ_area_ts)

unique_ds <- unique(occ_area_ts$dataset_id)
length(unique_ds)

ds_info <- dataset(datasetid = unique_ds[3])
ds_info$title

There are 4 datasets with at least 3 unique years, totalizing 29 sites (note that some may be repeated between the datasets). We can see that one of them is a standardized survey with abundance information:

**Reef Check Med - key Mediterranean marine species 2001-2020:** https://obis.org/dataset/b52cf4cf-6222-4318-9e43-355026336e03

In [ ]:
med_ds <- occurrence(datasetid = "b52cf4cf-6222-4318-9e43-355026336e03", mof = T)

med_ds_ext <- unnest_extension(med_ds, extension = "MeasurementOrFact",
                               fields = "eventDate")

med_ds_ext <- med_ds_ext |> 
    left_join(med_ds[,c("occurrenceID", "decimalLongitude", "decimalLatitude")]) |> 
    mutate(eventDate = as.Date(eventDate)) |> 
    # We will "grid" it, just for this practical
    group_by(decimalLongitude = round(decimalLongitude, 2),
             decimalLatitude = round(decimalLatitude, 2)) |> 
    mutate(site_id = cur_group_id()) |> 
    filter(measurementType == "Abundance category of biological entity specified elsewhere")

In [ ]:
# Plot one site
med_ds_ext_filt <- med_ds_ext |> 
    filter(site_id == 30) |> 
    group_by(eventDate, measurementValue) |> 
    count()

ggplot(med_ds_ext_filt) +
    geom_bar(aes(x = measurementValue, y = n, fill = measurementValue), stat = "identity") +
    scale_fill_brewer("Blues") +
    facet_wrap(~eventDate, nrow = 3) + 
    ylab("Unique occurrences with this category") + xlab("Date") +
    ggtitle("Abundance categories over time") + theme_light() +
    theme(panel.grid.major.x = element_blank(),
          panel.grid.minor.y = element_blank())

# • Category 0: 0 specimens (absent)
# • Category 1: 1 specimen (isolated specimen)
# • Category 2: 2 specimens (some scattered specimens)
# • Category 3: 3–5 specimens (several scattered specimens)
# • Category 4: 6–10 specimens (a crowded area)
# • Category 5: 11–50 specimens (some crowded areas)
# • Category 6: > 50 specimens (several crowded areas)

A special case is that of tracking data. You can learn more about tracking data in OBIS in the Ocean Tracking Network [website](https://oceantrackingnetwork.org/). Those datasets contains multiple measurements for the same organism, each at a time point.

Let's download one of those [datasets from OTN](https://obis.org/dataset/78bf6b7f-555c-4bf7-8d81-a766c5bc736e).

In [ ]:
otn_dataset <- occurrence(datasetid = "78bf6b7f-555c-4bf7-8d81-a766c5bc736e")

otn_dataset |> 
    select(scientificName, organismID, organismName, eventDate) |> 
    slice_head(n = 5)

As you can see, each organism has a unique ID and name that we can use to track all records pertaining to that animal.

In [ ]:
# Get the number of records by organism
ind_recs <- otn_dataset |> 
    group_by(organismID, eventDate, decimalLongitude, decimalLatitude) |> 
    distinct(.keep_all = T) |> 
    ungroup() |> group_by(organismID) |> 
    summarise(total = n()) |> 
    arrange(desc(total))

# Select the one with more records
blue_shark1 <- otn_dataset |> 
    filter(organismID == ind_recs$organismID[1])
head(blue_shark1)

# Arrange by date and plot
blue_shark1 <- blue_shark1 |> 
    group_by(eventDate, decimalLongitude, decimalLatitude) |> 
    distinct() |> 
    ungroup() |> 
    mutate(eventDate = lubridate::as_date(eventDate)) |> 
    arrange(eventDate)

plot(blue_shark1[,c("decimalLongitude", "decimalLatitude")])


In [ ]:

# Create an animated plot of movement
wrld <- rnaturalearth::ne_countries(returnclass = "sf")
wrld <- sf::st_as_sfc(wrld[,1])
lims_lon <- range(blue_shark1$decimalLongitude)
lims_lat <- range(blue_shark1$decimalLatitude)
lims_lon <- lims_lon + c(-0.1, 0.1)
lims_lat <- lims_lat + c(-0.1, 0.1)
coords <- c("decimalLongitude", "decimalLatitude")

png_path <- file.path(tempdir(), "frame%03d.png")
png(png_path)
for (i in seq_len(nrow(blue_shark1))) {
    plot(wrld, xlim = lims_lon, ylim = lims_lat, col = "grey70", main = blue_shark1$eventDate[i])
    points(blue_shark1[i,coords], col = "#044c98", pch = 20, cex = 2)
    if (i == 2) {
        points(blue_shark1[(i-1),coords], col = "#044b9889", pch = 20, cex = 2)
    } else if (i == 3) {
        points(blue_shark1[(i-1),coords], col = "#044b9889", pch = 20, cex = 2)
        points(blue_shark1[(i-2),coords], col = "#044b982f", pch = 20, cex = 2)
    } else if (i > 3) {
        points(blue_shark1[(i-1),coords], col = "#044b9889", pch = 20, cex = 2)
        points(blue_shark1[(i-2),coords], col = "#044b982f", pch = 20, cex = 2)
        points(blue_shark1[(i-3),coords], col = "#044b9812", pch = 20, cex = 2)
    }
}
dev.off()
png_files <- sprintf(png_path, seq_len(nrow(blue_shark1)))
gif_file <- tempfile(fileext = ".gif")
gifski::gifski(png_files, gif_file, delay = .1)
unlink(png_files)
utils::browseURL(gif_file)
# Note: if the file don't open, add this line, run again, and then you will
# see the GIF on your files explorer on the left side of Colab (refresh if needed) 
# Then, just download it to your computer.
file.copy(gif_file, "movement.gif")

#### To learn more

- OBIS manual: https://manual.obis.org/
- OBIS tutorials: https://resources.obis.org/tutorials/
- OBIS Discourse: https://obis.discourse.group/